# COMPAS Analysis

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv("./data/compas-scores-two-years.csv")
len(raw_data)

7214

In [3]:
# filter data
df = raw_data.loc[
    (raw_data['days_b_screening_arrest'] <= 30) &
    (raw_data['days_b_screening_arrest'] >= -30) &
    (raw_data['is_recid'] != -1) &
    (raw_data['c_charge_degree'] != "O") &
    (raw_data['score_text'] != "N/A")
]
len(df)

6172

In [4]:
# add "length of stay" column = date out - date in
df['length_of_stay'] = pd.to_numeric(pd.to_datetime(df['c_jail_out']) - pd.to_datetime(df['c_jail_in']))

In [5]:
#calculate the corr btw "length of stay" and "decile score"
print(df['length_of_stay'].corr(df['decile_score']))

0.20741201943031598


In [6]:
# demographic breakdown
df['age_cat'].value_counts()

25 - 45            3532
Less than 25       1347
Greater than 45    1293
Name: age_cat, dtype: int64

In [7]:
df['race'].value_counts()

African-American    3175
Caucasian           2103
Hispanic             509
Other                343
Asian                 31
Native American       11
Name: race, dtype: int64